# Land cover representation

This notebook compares the land cover within the entire study area to the land cover within a certain radius around sample locations.

#### Import statements

In [1]:
from database.models import Site
from landscape.models import LandCover
from geo.models import Boundary
from django.contrib.gis.db.models.functions import Intersection
from django.contrib.gis.db.models import Collect
from django.db import connection
from django.contrib.gis.geos import GEOSGeometry

from os import path
import numpy
import geopandas
import pyproj
from shapely.ops import unary_union, cascaded_union
from matplotlib import pyplot
%matplotlib inline

define the project coordinate system

In [2]:
austria_mgd = pyproj.Proj("+init=epsg:31254")

define the output filepath

In [3]:
output_filepath = ""

#### Function declarations

In [4]:
# returns a geopandas dataframe from a given PostGIS query
def get_geodataframe(queryset, modification=None, crs=austria_mgd):
    query = queryset.query.sql_with_params()
    if modification:
        query = (modification, query[1])
    return geopandas.read_postgis(query[0], connection, 
                                   geom_col='geometry', 
                                   params=query[1], 
                                   index_col='id',
                                   crs=crs)

In [5]:
# returns a geopandas dataframe containing the land cover intersected with a given geometry
def get_landcover(geometry, make_valid=False, crs=austria_mgd):
    
    if make_valid:
        intersection = """ST_Intersection(ST_MakeValid("landscape_landcover"."geometry"), %s)"""
    else:
        intersection = """ST_Intersection("landscape_landcover"."geometry", %s)"""
    
    query = """
        SELECT "landscape_landcover"."id", "landscape_landcover"."cover_type", 
        {0} AS "geometry" FROM "landscape_landcover" 
        WHERE ST_Intersects("landscape_landcover"."geometry", %s)
    """.replace('\n', '').format(intersection)

    return geopandas.read_postgis(query, connection, 
                                   geom_col='geometry', 
                                   params=["SRID={0};{1}".format(geometry.srid, geometry.wkt), 
                                           "SRID={0};{1}".format(geometry.srid, geometry.wkt)], 
                                   index_col='id',
                                   crs=crs)

## Get location land cover

define a list of radii defining the surrounding land cover

In [6]:
radii = [50, 100, 200, 500]

compute a geometry (unioned buffer around locations) for each radius

In [7]:
location_area_list = []
for radius in radii:
    location_areas = get_geodataframe(Site.objects.filter(id__lte=30)).buffer(radius)
    location_area = location_areas.iloc[0]
    for area in location_areas:
        location_area = location_area.union(area)
    # convert to geos multipolygon with srid
    location_area = GEOSGeometry(location_area.wkt)
    location_area.srid = 31254
    location_area_list.append(location_area)

query database

In [8]:
for location_area, radius in zip(location_area_list, radii):
    location_landcover = get_landcover(location_area)
    
    # export result as a csv file
    location_landcover.to_csv(path.join(output_filepath, "location_landcover_{0}m.csv".format(radius)))

## Get study area land cover

get the study area boundary geometry from the database

In [10]:
boundary = Boundary.objects.get(name="study area")

query database

In [11]:
study_landcover = get_landcover(boundary.geometry, make_valid=True)

# export result as a csv file
study_landcover.to_csv(path.join(output_filepath, "study_area_landcover.csv"))